In [3]:
import xgboost as xgb

ModuleNotFoundError: No module named 'xgboost'

In [4]:
!pip install xgboost

  Using cached xgboost-3.0.5-py3-none-win_amd64.whl.metadata (2.1 kB)
Using cached xgboost-3.0.5-py3-none-win_amd64.whl (56.8 MB)



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import xgboost as xgb


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb

# ======================
# 1) Load cleaned dataset
# ======================
df = pd.read_csv("../data/train_cleaned.csv")

X = df.drop("SalePrice", axis=1)
y = df["SalePrice"]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Training set:", X_train.shape, "Validation set:", X_val.shape)

# ======================
# 2) Convert to DMatrix (XGBoost format)
# ======================
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

# ======================
# 3) Set parameters
# ======================
params = {
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "eta": 0.05,
    "max_depth": 4,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "seed": 42
}

# ======================
# 4) Train with early stopping
# ======================
evals = [(dtrain, "train"), (dval, "eval")]

xgb_model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=1000,
    evals=evals,
    early_stopping_rounds=50,
    verbose_eval=100
)

# ======================
# 5) Evaluate model
# ======================
y_pred = xgb_model.predict(dval)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print("XGBoost RMSE:", rmse)

# ======================
# 6) Save model
# ======================
xgb_model.save_model("../src/xgb_model.json")
print("✅ XGBoost model saved to ../src/xgb_model.json")


Training set: (1168, 249) Validation set: (292, 249)
[0]	train-rmse:74257.85037	eval-rmse:84517.83606
[100]	train-rmse:14938.68727	eval-rmse:27285.54833
[200]	train-rmse:10860.16973	eval-rmse:25961.49385
[300]	train-rmse:8660.29046	eval-rmse:25760.02524
[313]	train-rmse:8435.28242	eval-rmse:25720.18593
XGBoost RMSE: 25720.18538035836
✅ XGBoost model saved to ../src/xgb_model.json
